In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 4.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/DW.Matrix'

/content/drive/My Drive/Colab Notebooks/DW.Matrix


In [0]:
ls data

men_shoes.csv  Men_Shoes_Prices_brands_manufactures.csv  Men_Shoes_Prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory = False)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth = 5)):
  X = df[feats].values
  Y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, Y, scoring = 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [0]:
df['brand_cat'] = df['brand'].factorize()[0]
run_model(['brand_cat'])

(-59.66740722193515, 5.86318651440948)

In [0]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
run_model(['brand_cat'], model)

(-59.15580015651201, 5.938098981190039)

In [0]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""

In [0]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-59.74991436028463, 5.857301054060102)

In [0]:
run_model(['brand_cat'], model)

(-59.16236629345697, 5.83857552622226)

In [0]:
df.features

0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
18337    [{"key":"Style","value":["Desert Boots"]},{"ke...
18338    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
18339    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18340    [{"key":"Gender","value":["Unisex"]},{"key":"F...
18341    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 18342, dtype: object

In [0]:
df.features.head()

0    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1    [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2    [{"key":"Gender","value":["Men"]},{"key":"Colo...
3    [{"key":"Gender","value":["Men"]},{"key":"Colo...
4    [{"key":"Gender","value":["Men"]},{"key":"Colo...
Name: features, dtype: object

In [0]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [0]:
str_dict = '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]'

literal_eval(str_dict)

[{'key': 'Gender', 'value': ['Men']},
 {'key': 'Shoe Size', 'value': ['M']},
 {'key': 'Shoe Category', 'value': ["Men's Shoes"]},
 {'key': 'Color', 'value': ['Multicolor']},
 {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']},
 {'key': 'Brand', 'value': ['Josmo']}]

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan':
    return output_dict
  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    output_dict[item['key'].lower().strip()] = item['value'][0].lower().strip() #<--- małe litery plus usuwamy spacje.
  return output_dict

df['features_parsed'] = df['features'].map(parse_features)

In [53]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [54]:
df.features_parsed

0        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1        {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2        {'gender': 'men', 'color': 'black', 'shipping ...
3        {'gender': 'men', 'color': 'black', 'shipping ...
4        {'gender': 'men', 'color': 'black', 'shipping ...
                               ...                        
18337    {'style': 'desert boots', 'pattern': 'solid', ...
18338    {'shoe size': '9.5', 'gender': 'men', 'shoe ca...
18339    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18340    {'gender': 'unisex', 'frame style': 'wrap', 'p...
18341    {'gender': 'unisex', 'frame style': 'wrap', 'p...
Name: features_parsed, Length: 18342, dtype: object

In [56]:
keys = set()
df.features_parsed.map(lambda x: keys.update(x.keys()))
len(keys)

476

In [92]:
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [59]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_general warranty', 'feat_video game platform',
       'feat_enhanced visibility (reflective)', 'feat_machine washable',
       'feat_heat zones', 'feat_construction', 'feat_shoe category',
       'feat_chest', 'feat_polarized?', 'feat_weight'],
      dtype='object', length=526)

In [60]:
len(df.columns)

526

In [61]:
df[df.feat_athlete.isnull()].shape #<--- łapie inne, niż feat_athlete.

(18334, 526)

In [62]:
df.shape[0] #<--- Tutaj widzimy, że 8 razy tylko występuje feat_athlete (jest niepuste)

18342

In [64]:
(df[df.feat_athlete.isnull() == False].shape[0] / df.shape[0]) * 100

0.04361574528404754

In [0]:
keys_stat = {}

for key in keys:
  keys_stat[key] = (df[df[get_name_feat(key)].isnull() == False].shape[0] / df.shape[0]) * 100

In [68]:
keys_stat

{'100% adidas authentic': 0.0054519681605059425,
 'accessory type': 0.14175117217315453,
 'adidas': 0.0054519681605059425,
 'adjustable': 0.34347399411187435,
 'age': 0.6378802747791953,
 'age end': 0.4416094210009814,
 'age gender group': 0.12539526769163667,
 'age group': 27.554247083197037,
 'age range': 0.5451968160505942,
 'age segment': 0.14175117217315453,
 'age start': 0.4416094210009814,
 'airport friendly': 0.0054519681605059425,
 'alarm': 0.23443463090175554,
 'amazonbestsellersrank': 0.05997164976556537,
 'animal type': 0.0054519681605059425,
 'antiscratch lens coating': 0.0054519681605059425,
 'applicable': 0.02180787264202377,
 'arm': 0.6215243702976775,
 'article': 0.0054519681605059425,
 'assembled in country of origin': 2.191691200523389,
 'assembled product dimensions (l x w x h)': 10.26605604623269,
 'assembled product weight': 1.5483589575836878,
 'athlete': 0.04361574528404754,
 'atpv arc rating': 0.0054519681605059425,
 'attachment': 0.03271180896303566,
 'audienc

In [74]:
something = {k: v for k, v in keys_stat.items() if v > 30}
something

{'brand': 48.46254497873733,
 'color': 47.62294188201941,
 'gender': 50.00545196816051,
 'manufacturer part number': 36.130192999672886,
 'material': 34.78900883218842}

In [0]:
def get_name_feat_cat(key):
  return get_name_feat(key) + '_cat'

for key in something.keys():
  df[get_name_feat_cat(key)] = df[get_name_feat(key)].factorize()[0]

#df['feat_brand_cat'] = df['feat_brand'].factorize()[0]

In [117]:
df['brand'] = df['brand'].map(lambda x: str(x).lower())
df[df.brand != df.feat_brand].shape

(9496, 536)

In [118]:
df[df.brand == df.feat_brand].shape

(8846, 536)

In [119]:
df[df.brand != df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
21,rubies,generic
22,rubies,generic
23,rubies,generic
24,unbranded,NaN
31,american fighter,NaN


In [120]:
df[df.brand == df.feat_brand][['brand', 'feat_brand']].head()

,brand,feat_brand
0,josmo,josmo
1,josmo,josmo
2,servus by honeywell,servus by honeywell
3,servus by honeywell,servus by honeywell
4,servus by honeywell,servus by honeywell


In [121]:
model = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
run_model(['brand_cat'], model), run_model(['brand_cat'])

((-59.16236629345697, 5.83857552622226),
 (-59.74991436028463, 5.857301054060102))

In [129]:
feats = [get_name_feat_cat(key) for key in something.keys()] + ['brand_cat']
run_model(feats, model)

(-58.62260568147163, 6.097897489460527)

In [123]:
X = df[feats].values
Y = df.prices_amountmin.values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, Y)

perm = PermutationImportance(m, random_state = 1).fit(X, Y);
eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.3135 ± 0.0123,brand_cat
0.0876 ± 0.0172,feat_material_cat
0.0546 ± 0.0088,feat_color_cat
0.0385 ± 0.0037,feat_gender_cat
0.0201 ± 0.0003,feat_brand_cat
0.0011 ± 0.0002,feat_manufacturer part number_cat


In [132]:
feats_prime = feats.copy()
feats_prime.remove('feat_manufacturer part number_cat')
run_model(feats, model), run_model(feats_prime, model)

((-58.62260568147163, 6.097897489460527),
 (-58.57903299057339, 6.0604371637289045))

In [133]:
X = df[feats_prime].values
Y = df.prices_amountmin.values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, Y)

perm = PermutationImportance(m, random_state = 1).fit(X, Y);
eli5.show_weights(perm, feature_names = feats_prime)

Weight,Feature
0.3127 ± 0.0126,brand_cat
0.0850 ± 0.0225,feat_material_cat
0.0473 ± 0.0088,feat_color_cat
0.0414 ± 0.0062,feat_gender_cat
0.0209 ± 0.0013,feat_brand_cat


In [151]:
feats_prime_zwei = feats_prime.copy()
feats_prime_zwei.remove('feat_brand_cat')
run_model(feats, model), run_model(feats_prime, model), run_model(feats_prime_zwei, model)

((-58.62260568147163, 6.097897489460527),
 (-58.57903299057339, 6.0604371637289045),
 (-58.59886824598904, 6.056904320492506))

In [152]:
df.brand.value_counts(normalize = True)

nike                  0.096881
puma                  0.033202
ralph lauren          0.032058
vans                  0.021045
new balance           0.020227
                        ...   
mancini leather       0.000055
es                    0.000055
coronado              0.000055
deadman wonderland    0.000055
nike sb               0.000055
Name: brand, Length: 1732, dtype: float64

In [139]:
df[df.brand == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
df['feat_sport_cat'] = df['feat_sport'].factorize()[0]
df['feat_style_cat'] = df['feat_style'].factorize()[0]

In [157]:
feats_prime_drei = feats_prime_zwei.copy()
feats_prime_drei.append('feat_style_cat')
#feats_prime_drei.append('feat_sport_cat')
feats_prime_drei

['feat_color_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'brand_cat',
 'feat_style_cat']

In [158]:
run_model(feats, model), run_model(feats_prime, model), run_model(feats_prime_zwei, model), run_model(feats_prime_drei, model)

((-58.62260568147163, 6.097897489460527),
 (-58.57903299057339, 6.0604371637289045),
 (-58.59886824598904, 6.056904320492506),
 (-58.61492174282188, 6.14404855246087))

In [159]:
X = df[feats_prime_drei].values
Y = df.prices_amountmin.values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, Y)

perm = PermutationImportance(m, random_state = 1).fit(X, Y);
eli5.show_weights(perm, feature_names = feats_prime_drei)

Weight,Feature
0.3094 ± 0.0137,brand_cat
0.0594 ± 0.0092,feat_material_cat
0.0592 ± 0.0105,feat_color_cat
0.0444 ± 0.0040,feat_gender_cat
0.0216 ± 0.0103,feat_style_cat


In [160]:
df[df.brand == 'nike'].features_parsed.sample(5).values

array([{'sport': 'soccer', 'condition': 'new with box', 'type': 'ag cleats'},
       {'sport': 'running shoes', 'occasion': 'running shoes', 'material': 'synthetic-and-mesh', 'gender': 'men', 'shoe size': '7.5 us men', 'shoe category': "men's shoes", 'assembled product dimensions (l x w x h)': '12.00 x 8.00 x 6.00 inches', 'fabric content': 'synthetic-and-mesh', 'color': 'green', 'model': '631047 340', 'shoe closure': 'lace-up', 'casual & dress shoe style': 'running shoes', 'manufacturer part number': '631047 340', 'brand': 'nike', 'shoe width': 'd(m)'},
       {'country/region of manufacture': 'china', 'mpn': '684860-003', 'color': 'silver'},
       {'main color': 'black', 'country/region of manufacture': 'china'},
       {'gender': 'men', 'shoe category': 'basketball', 'brand': 'nike', 'manufacturer_part_number': '130690-001', 'fabric content': '100% leather', 'color': 'black/varsity red-white', 'model': '130690-001'}],
      dtype=object)

In [163]:
df['feat_age group'].value_counts()

adult               4563
men                  350
child                 77
men's                 33
unisex                 6
mens                   4
infant                 4
toddler                4
boys'                  3
women                  2
men||women             2
women ,�� unisex       2
youth                  2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [0]:
for key in keys:
  df[get_name_feat_cat(key)] = df[get_name_feat(key)].factorize()[0]

In [167]:
feats_cat = [x for x in df.columns if '_cat' in x]
feats_cat

['brand_cat',
 'feat_catalog',
 'feat_color_cat',
 'feat_manufacturer part number_cat',
 'feat_brand_cat',
 'feat_gender_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_character_cat',
 'feat_materials_cat',
 'feat_profession_cat',
 'feat_disclosures_cat',
 'feat_bridge width_cat',
 'feat_safety apparel product type_cat',
 'feat_product type_cat',
 'feat_fits over eyeglasses_cat',
 'feat_country//region of manufacture_cat',
 'feat_msrp_cat',
 'feat_gps_cat',
 'feat_auto shut-off_cat',
 'feat_clothing category_cat',
 'feat_country/ organization_cat',
 'feat_genre_cat',
 'feat_digital camera_cat',
 'feat_thick, warm, cozy_cat',
 'feat_case type_cat',
 'feat_arm_cat',
 'feat_motion control_cat',
 'feat_theme_cat',
 'feat_alarm_cat',
 'feat_front_cat',
 'feat_band type_cat',
 'feat_sleeve length_cat',
 'feat_waist_cat',
 'feat_fits model_cat',
 'feat_manufacturer_part_number_cat',
 'feat_fit:_cat',
 'feat_boxed-product dimensions_cat',
 'feat_fishing_cat',
 'feat_a

In [172]:
feats_fear = list(set(feats + ['feat_recommended location_cat',
 'feat_case finish_cat',
 'feat_technician_cat',
 'feat_subdial_cat',
 'feat_model number_cat',
 'feat_bracelet style_cat',
 'feat_shape_cat',
 'feat_frame material:_cat',
 'feat_heart rate monitor_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_insulated_cat',
 'feat_domestic shipping_cat',
 'feat_features & fastening_cat',
 'feat_width_cat',
 'feat_hood_cat',
 'feat_heel height_cat',
 'feat_parts_cat',
 'feat_tactical_cat',
 'feat_number of heat settings_cat',
 'feat_colour_cat',
 'feat_watch style_cat',
 'feat_hat style_cat',
 'feat_shirt size_cat',
 'feat_weather resistant_cat',
 'feat_movement_cat',
 'feat_color family_cat']))


X = df[feats_fear].values
Y = df.prices_amountmin.values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, Y)

perm = PermutationImportance(m, random_state = 1).fit(X, Y);
eli5.show_weights(perm, feature_names = feats_fear)

Weight,Feature
0.3088 ± 0.0147,brand_cat
0.0837 ± 0.0115,feat_material_cat
0.0466 ± 0.0014,feat_color_cat
0.0203 ± 0.0039,feat_gender_cat
0.0168 ± 0.0022,feat_brand_cat
0.0053 ± 0.0003,feat_movement_cat
0.0048 ± 0.0003,feat_watch style_cat
0.0037 ± 0.0004,feat_frame material:_cat
0.0008 ± 0.0001,feat_shape_cat
0.0005 ± 0.0002,feat_manufacturer part number_cat


In [0]:
#feat_resizable_cat
#feat_condition_cat
#To można dorzucić i liczyć, czy wyjdzie lepiej, czy nie?


In [173]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [0]:
# lbs == 453.592
# ounces == 28.3495
# pound == 453.5924

In [0]:
def normalize_to_gram(x):
  if str(x) == 'nan':
    return -1
  
  value, unit = x.lower().split(' ')
  value = float(value)

  if unit == 'lb' or unit == 'lbs' or unit == 'pounds' or unit == 'pound':
    value *= 453.5924
  elif unit == 'ounces':
    value *= 28.3495
  elif unit == 'kg':
    value *= 1000
  elif unit == 'g':
    pass
  else:
    print('ERROR: ', unit)
  return value

df['weight_norm'] = df['weight'].map(normalize_to_gram).factorize()[0]

In [0]:
temp = ['feat_condition_cat', 'weight_norm']

featts = feats_prime.copy()
featts += temp
RESULT = run_model(feats_prime, model), run_model(featts, model)

In [201]:
X = df[featts].values
Y = df.prices_amountmin.values

m = RandomForestRegressor(max_depth = 5, n_estimators = 100, random_state = 0)
m.fit(X, Y)

print("The first result does not contain 'feat_condition_cat' and 'weight_norm'. It looks like this:\n", RESULT, "\nThe last picture: ")

perm = PermutationImportance(m, random_state = 1).fit(X, Y);
eli5.show_weights(perm, feature_names = featts)

The first result does not contain 'feat_condition_cat' and 'weight_norm'. It looks like this:
 ((-58.57903299057339, 6.0604371637289045), (-58.55473257572487, 6.065107082259811)) 
The last picture: 


Weight,Feature
0.3068 ± 0.0075,brand_cat
0.0641 ± 0.0049,feat_material_cat
0.0468 ± 0.0013,feat_color_cat
0.0403 ± 0.0045,feat_gender_cat
0.0205 ± 0.0019,feat_brand_cat
0.0086 ± 0.0058,feat_condition_cat
0.0011 ± 0.0016,weight_norm


In [0]:
!git add Matrix_One/Day5.ipynb

In [0]:
!git config --global user.email "urbas.kamil@gmail.com"
!git config --global user.name "Kamil"

In [0]:
!git commit -m "Read Men's Shoe Prices dataset from data.world and easy machine learning model"

In [0]:
!git push -u origin master